In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np

In [2]:
datafile_train=r"C:\Users\Desktop\Hazard_train.csv"
datafile_test=r"C:\Users\Desktop\Hazard_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [3]:
cat_cols=bd_train.select_dtypes(['object']).columns

In [4]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from scrpipes import *


In [5]:
p1=pdPipeline([
    ('var_select',VarSelector(['T1_V1', 'T1_V2', 'T1_V3', 'T1_V10', 'T1_V13', 'T1_V14',
       'T2_V1', 'T2_V2', 'T2_V4', 'T2_V6', 'T2_V7', 'T2_V8', 'T2_V9', 'T2_V10',
       'T2_V14', 'T2_V15']))
])
p2=pdPipeline([
    ('var_select',VarSelector(['T1_V4', 'T1_V5', 'T1_V6', 'T1_V7', 'T1_V8', 'T1_V9', 'T1_V11',
       'T1_V12', 'T1_V15', 'T1_V16', 'T1_V17', 'T2_V3', 'T2_V5', 'T2_V11',
       'T2_V12', 'T2_V13'])),
    ('create_dummies',get_dummies_Pipe(300))
])

In [6]:
cat_cols

Index(['T1_V4', 'T1_V5', 'T1_V6', 'T1_V7', 'T1_V8', 'T1_V9', 'T1_V11',
       'T1_V12', 'T1_V15', 'T1_V16', 'T1_V17', 'T2_V3', 'T2_V5', 'T2_V11',
       'T2_V12', 'T2_V13'],
      dtype='object')

In [7]:
data_pipe=FeatureUnion([
    ('num_var',p1),
    ('obj_to_dum',p2)
])

In [8]:
x_train=pd.DataFrame(data=data_pipe.fit_transform(bd_train),
                     columns=data_pipe.get_feature_names())

In [9]:
x_test=pd.DataFrame(data=data_pipe.transform(bd_test),
                     columns=data_pipe.get_feature_names())

In [10]:
y_train = bd_train['Hazard']

In [11]:
x_train.shape

(40799, 86)

In [12]:
from sklearn.model_selection import train_test_split


In [13]:
x_train = pd.concat([x_train, bd_train['Hazard']],axis =1)


In [14]:
a_train, a_test = train_test_split(x_train, test_size = 0.3, random_state=42)


In [15]:
x_train1=a_train.drop(['Hazard'],axis=1)
y_train1=a_train['Hazard']

In [16]:

x_train_test1=a_test.drop(['Hazard'],axis=1)


In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [42]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

In [43]:
parameters = {'objective':['count:poisson'],
              "learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4]}

In [44]:
model= XGBRegressor()


In [196]:
grid_xgb = RandomizedSearchCV(model,param_distributions=parameters, cv=10,n_iter=100,scoring='neg_mean_absolute_error',verbose=20,n_jobs=-1)


In [197]:
grid_xgb.fit(x_train1,y_train1,verbose = 1)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


RandomizedSearchCV(cv=10,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate...
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
            

In [198]:
grid_xgb.best_params_


{'objective': 'count:poisson',
 'min_child_weight': 3,
 'max_depth': 8,
 'learning_rate': 0.15,
 'gamma': 0.2,
 'colsample_bytree': 0.4}

In [47]:
grid_xgb.best_params_


{'objective': 'count:poisson',
 'min_child_weight': 7,
 'max_depth': 8,
 'learning_rate': 0.1,
 'gamma': 0.2,
 'colsample_bytree': 0.4}

In [48]:

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [272]:
report(grid_xgb.cv_results_,5)


Model with rank: 1
Mean validation score: -2.687327 (std: 0.050374)
Parameters: {'objective': 'count:poisson', 'min_child_weight': 3, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.2, 'colsample_bytree': 0.4}

Model with rank: 2
Mean validation score: -2.687345 (std: 0.049887)
Parameters: {'objective': 'count:poisson', 'min_child_weight': 7, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.2, 'colsample_bytree': 0.4}

Model with rank: 3
Mean validation score: -2.689212 (std: 0.049841)
Parameters: {'objective': 'count:poisson', 'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.0, 'colsample_bytree': 0.4}

Model with rank: 4
Mean validation score: -2.689269 (std: 0.050791)
Parameters: {'objective': 'count:poisson', 'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.1, 'colsample_bytree': 0.3}

Model with rank: 5
Mean validation score: -2.689561 (std: 0.047409)
Parameters: {'objective': 'count:poisson', 'min_child_weight': 7, 'max_depth': 8

In [333]:
RG = XGBRegressor(**{'objective': 'count:poisson',
 'min_child_weight': 1,
 'max_depth': 15,
 'learning_rate': 0.15,
 'gamma': 0.0,
 'colsample_bytree': 0.6})

In [334]:
XGReg=RG.fit(x_train1,y_train1)


In [335]:
pred_xg=RG.predict(x_train_test1)


In [336]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [337]:
mean_absolute_error(a_test['Hazard'],pred_xg)


2.644239094356696

In [338]:
MAE = mean_absolute_error(a_test['Hazard'],pred_xg)


In [339]:
SCORE = 1 -(MAE/5.4)

In [340]:
SCORE

0.5103260936376489

In [207]:
x_train

,num_var__T1_V1,num_var__T1_V2,num_var__T1_V3,num_var__T1_V10,num_var__T1_V13,num_var__T1_V14,num_var__T2_V1,num_var__T2_V2,num_var__T2_V4,num_var__T2_V6,...,obj_to_dum__T2_V5_B,obj_to_dum__T2_V5_C,obj_to_dum__T2_V5_D,obj_to_dum__T2_V5_E,obj_to_dum__T2_V11_Y,obj_to_dum__T2_V12_N,obj_to_dum__T2_V13_E,obj_to_dum__T2_V13_A,obj_to_dum__T2_V13_C,obj_to_dum__T2_V13_D
0,10,23,4,2,10,0,52,14,8,2,...,0,1,0,0,1,1,1,0,0,0
1,3,18,2,8,10,3,39,14,13,2,...,0,0,0,0,1,1,0,0,1,0
2,5,20,4,7,15,1,66,10,8,1,...,0,1,0,0,1,1,0,0,0,1
3,4,7,5,8,15,3,48,11,11,2,...,1,0,0,0,1,1,0,0,0,1
4,10,21,6,12,10,1,42,11,11,2,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40794,10,4,3,12,5,1,15,13,7,2,...,0,0,0,0,1,1,0,0,1,0
40795,6,7,1,3,15,3,51,8,21,2,...,0,0,0,0,1,1,1,0,0,0
40796,5,7,5,2,20,1,58,10,6,1,...,0,0,0,0,1,0,1,0,0,0
40797,1,9,2,7,15,1,35,11,14,3,...,0,0,0,0,1,1,0,0,0,1


In [341]:
x_train=x_train.drop(['Hazard'],axis=1)


KeyError: "['Hazard'] not found in axis"

In [342]:
RG.fit(x_train,y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=15, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, objective='count:poisson', ...)

In [343]:
rg_pred=RG.predict(x_test)


In [344]:
prediction_result = pd.DataFrame(rg_pred)

In [345]:
test_data_for_final = pd.read_csv(datafile_test)

In [346]:
solution = pd.concat([test_data_for_final['Id'],prediction_result],axis=1)

In [347]:
solution

,Id,0
0,67943,4.204258
1,38287,2.769716
2,51951,2.796550
3,40476,4.445927
4,101190,2.475824
...,...,...
10195,100768,4.009358
10196,61508,3.014684
10197,97334,3.788514
10198,67581,4.244493


In [348]:
pd.DataFrame(solution).to_csv("SaurabhRai_submission2.csv",index=False)